<a href="https://colab.research.google.com/github/amankhullar/cs20/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> CS20 Assignment 1 </h1> </center>
## Problem 1: Op is all you need

In [0]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVL'] = '2'

import tensorflow as tf


In [0]:
sess = tf.InteractiveSession()

x = tf.random_uniform([])
y = tf.random_uniform([])

op = tf.cond(x > y, lambda : x + y, lambda : x - y)

print(sess.run(op))

1.1196562


In [0]:
x = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "x")
y = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "y")

# Using a tf.Tensor as a Python 'bool' is not allowed
# out = tf.zeros(shape = [1])

# if tf.less(x, y):
#     out = x + y
# elif tf.greater(x, y):
#     out = x - y
# else:
#     out = 0
    
def f1():
    return x + y

def f2():
    return x - y

def f3():
    return tf.zeros(shape = [1])

out = tf.case({tf.less(x, y) : f1, tf.greater(x, y) : f2}, default = f3, name = 'output', exclusive = True)
print(sess.run(out))

0.27322483


In [0]:
x = tf.constant([[0, -2, -1], [0, 1, 2]])
y = tf.zeros_like(x)

out = tf.equal(x, y, name = 'equal_output')

print(sess.run(out))

[[ True False False]
 [ True False False]]


In [0]:
sess = tf.InteractiveSession()

data = tf.constant([29.05088806,  27.61298943,  31.19073486,  29.35532951,
                     30.97266006,  26.67541885,  38.08450317,  20.74983215,
                     34.94445419,  34.45999146,  29.06485367,  36.01657104,
                     27.88236427,  20.56035233,  30.20379066,  29.51215172,
                     33.71149445,  28.59134293,  36.05556488,  28.66994858])

out = tf.gather(data, tf.where(tf.greater(data, 30)))

print(sess.run(out))

[[31.190735]
 [30.97266 ]
 [38.084503]
 [34.944454]
 [34.45999 ]
 [36.01657 ]
 [30.20379 ]
 [33.711494]
 [36.055565]]


In [0]:
# Numpy method of creating the diagonal matrix

# data = np.arange(1,7)
# out = np.diag(data)

# out_tf = tf.constant(out)

# print(sess.run(out_tf))

#Tensorflow method

data = tf.range(start = 1, limit = 7, name = 'range')
out = tf.linalg.diag(data, name = 'diagonal')

print(sess.run(out))

[[1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


In [0]:
data = tf.random_normal(shape = [10, 10], name = 'matrix')

out = tf.matrix_determinant(data, name = 'determinant')

print(sess.run(out))

-66.201225


In [0]:
x = tf.constant([5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9])

out, idx = tf.unique(x)

print(sess.run(out))

[ 5  2  3 10  6  4  1  0  9]


In [0]:
x = tf.random_normal(shape = [300], name = 'x')
y = tf.random_normal(shape = [300], name = 'y')

out = tf.cond(tf.reduce_mean(x - y) < 0, lambda : tf.losses.mean_squared_error(labels = x, predictions = y), lambda : tf.reduce_sum(tf.abs(x - y)))

print(sess.run(out))

2.150824


## Problem 2: Logistic Regression

### Utils

In [0]:
import os
import gzip
import shutil
import struct
import urllib

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

def huber_loss(labels, predictions, delta=14.0):
    residual = tf.abs(labels - predictions)
    def f1(): return 0.5 * tf.square(residual)
    def f2(): return delta * residual - 0.5 * tf.square(delta)
    return tf.cond(residual < delta, f1, f2)

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

def read_birth_life_data(filename):
    """
    Read in birth_life_2010.txt and return:
    data in the form of NumPy array
    n_samples: number of samples
    """
    text = open(filename, 'r').readlines()[1:]
    data = [line[:-1].split('\t') for line in text]
    births = [float(line[1]) for line in data]
    lifes = [float(line[2]) for line in data]
    data = list(zip(births, lifes))
    n_samples = len(data)
    data = np.asarray(data, dtype=np.float32)
    return data, n_samples

def download_one_file(download_url, 
                    local_dest, 
                    expected_byte=None, 
                    unzip_and_remove=False):
    """ 
    Download the file from download_url into local_dest
    if the file doesn't already exists.
    If expected_byte is provided, check if 
    the downloaded file has the same number of bytes.
    If unzip_and_remove is True, unzip the file and remove the zip file
    """
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' %local_dest)
    else:
        print('Downloading %s' %download_url)
        local_file, _ = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' %local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3],'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')

def download_mnist(path):
    """ 
    Download and unzip the dataset mnist if it's not already downloaded 
    Download from http://yann.lecun.com/exdb/mnist
    """
    safe_mkdir(path)
    url = 'http://yann.lecun.com/exdb/mnist'
    filenames = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
    expected_bytes = [9912422, 28881, 1648877, 4542]

    for filename, byte in zip(filenames, expected_bytes):
        download_url = os.path.join(url, filename)
        local_dest = os.path.join(path, filename)
        download_one_file(download_url, local_dest, byte, True)

def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test

def get_mnist_dataset(batch_size):
    # Step 1: Read in data
    mnist_folder = 'data/mnist'
    download_mnist(mnist_folder)
    train, val, test = read_mnist(mnist_folder, flatten=False)

    # Step 2: Create datasets and iterator
    train_data = tf.data.Dataset.from_tensor_slices(train)
    train_data = train_data.shuffle(10000) # if you want to shuffle your data
    train_data = train_data.batch(batch_size)

    test_data = tf.data.Dataset.from_tensor_slices(test)
    test_data = test_data.batch(batch_size)

    return train_data, test_data
    
def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    plt.imshow(image, cmap='gray')
    plt.show()

### Tensorboard for Colab

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc = TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://d5c26176.ngrok.io


### Logistic Regression

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
import time

#Defining the hyperparameters

learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

#Step 1: Read in data
mnist_folder = 'data/mnist'
download_mnist(mnist_folder)
train, val, test = read_mnist(mnist_folder, flatten = True)

#Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

#Create one iterator and initialize it with different datasetsvim
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

#Step 3: create weights and bias
w = tf.get_variable(name = 'weights', shape = [784, 10], initializer = tf.random_normal_initializer(mean = 0, stddev = 0.01))
b = tf.get_variable(name = 'bias', shape = [1, 10], initializer = tf.zeros_initializer())

#Step 4: build model
logits = tf.matmul(img, w) + b

#Step 5: define the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = label, logits = logits, name = 'entropy')
loss = tf.reduce_mean(entropy, name = 'loss')

#Step 6: define training operation
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds,1), tf.argmax(label,1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./Graph/logreg', tf.get_default_graph())
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    #train the model number of epochs times
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Toal time : {0} seconds'.format(time.time() - start_time))
        
    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists
Average loss epoch 0: 0.36396381806495576
Average loss epoch 1: 0.29472465308946233
Average loss epoch 2: 0.283943564791319
Average loss epoch 3: 0.2781646569107854
Average loss epoch 4: 0.2736546872660171
Average loss epoch 5: 0.2742128196670566
Average loss epoch 6: 0.2704728376900041
Average loss epoch 7: 0.2655832806298899
Average loss epoch 8: 0.2627160980777685
Average loss epoch 9: 0.263606508318768
Average loss epoch 10: 0.26379387650725455
Average loss epoch 11: 0.2616857203633286
Average loss epoch 12: 0.26098569477366845
Average loss epoch 13: 0.2592407028688941
Average loss epoch 14: 0.2581491239022377
Average loss epoch 15: 0.25816859088318295
Average loss epoch 16: 0.25917671565399614
Average loss epoch 17: 0.2572768713325955
Average loss epoch 18: 0.256737866651180

### Improved accuracy of Logistic Regression 

In [0]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import time

#Defining the hyperparameters

learning_rate = 0.01
batch_size = 128
n_epochs = 50
n_train = 60000
n_test = 10000
n_hidden = 500

#Step 1: Read Data
mnist_folder = 'data/mnist'
download_mnist(mnist_folder)
train, val, test = read_mnist(mnist_folder, flatten = True)

#Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

#Create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

#Step 3: create weights and bias
w1 = tf.get_variable(name = 'weights_1', shape = (784, n_hidden), initializer = tf.random_normal_initializer(mean = 0, stddev = 0.01))
b1 = tf.get_variable(name = 'bias_1', shape = (1, n_hidden), initializer = tf.zeros_initializer())

#Parameters for the hidden layer and the output layer
w2 = tf.get_variable(name = 'weights_2', shape = (n_hidden, 10), initializer = tf.random_normal_initializer(mean = 0, stddev = 0.01))
b2 = tf.get_variable(name = 'bias_2', shape = (1, 10), initializer = tf.zeros_initializer())

#Step 4: build model
z1 = tf.matmul(img, w1) + b1
h1 = tf.nn.sigmoid(z1, name = 'sigmoid_layer')

logits = tf.matmul(h1, w2) + b2

#Step 5: define the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = label, logits = logits, name = 'entropy')
loss = tf.reduce_mean(entropy, name = 'loss')

#Step 6: define the training operation
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#Step 7: calculate the accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./Graph/logreg_improved', tf.get_default_graph())

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    #train the model number of epochs times
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time : {0} seconds'.format(time.time() - start_time))

    #test model
    sess.run(test_init)
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/ n_test))
writer.close()

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists
Average loss epoch 0: 0.34232790607006053
Average loss epoch 1: 0.10143689684743105
Average loss epoch 2: 0.0669986465231104
Average loss epoch 3: 0.04679901247064388
Average loss epoch 4: 0.03501995945630898
Average loss epoch 5: 0.030108175212300792
Average loss epoch 6: 0.02773205312672829
Average loss epoch 7: 0.022330892543297606
Average loss epoch 8: 0.023507892807165896
Average loss epoch 9: 0.026051185270608946
Average loss epoch 10: 0.024742556574786904
Average loss epoch 11: 0.01712285785131741
Average loss epoch 12: 0.020678994558794902
Average loss epoch 13: 0.018964344293669645
Average loss epoch 14: 0.017029577171494968
Average loss epoch 15: 0.020649463424884152
Average loss epoch 16: 0.014399091447608784
Average loss epoch 17: 0.015060598406802624
Average loss epo

## Problem 3 : Word2Vec, Skip-Gram model

### Word2Vec_utils

In [0]:
from collections import Counter
import random
import os
import sys
sys.path.append('..')
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf


def read_data(file_path):
    """Read data into a list of tokens
    There should be 17,005,207 tokens
    """
    
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split() 
        return words
    
def build_vocab(words, vocab_size, visual_fld):
    """ Build vocabulary of VOCAB_SIZE most frequent words and write it to
    visualization/vocab.tsv
    """
    safe_mkdir(visual_fld)
    file = open(os.path.join(visual_fld, 'vocab.tsv'), 'w')
    
    dictionary = dict()
    count = [('UNK', -1)]
    index = 0
    count.extend(Counter(words).most_common(vocab_size - 1))
    
    for word, _ in count:
        dictionary[word] = index
        index += 1
        file.write(word + '\n')
    
    index_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    file.close()
    return dictionary, index_dictionary

def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

def generate_sample(index_words, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index, center in enumerate(index_words):
        context = random.randint(1, context_window_size)
        # get a random target before the center word
        for target in index_words[max(0, index - context): index]:
            yield center, target
        # get a random target after the center wrod
        for target in index_words[index + 1: index + context + 1]:
            yield center, target

def most_common_words(visual_fld, num_visualize):
    """ create a list of num_visualize most frequent words to visualize on TensorBoard.
    saved to visualization/vocab_[num_visualize].tsv
    """
    words = open(os.path.join(visual_fld, 'vocab.tsv'), 'r').readlines()[:num_visualize]
    words = [word for word in words]
    file = open(os.path.join(visual_fld, 'vocab_' + str(num_visualize) + '.tsv'), 'w')
    for word in words:
        file.write(word)
    file.close()

def batch_gen(download_url, expected_byte, vocab_size, batch_size, 
                skip_window, visual_fld):
    local_dest = 'data/text8.zip'
    download_one_file(download_url, local_dest, expected_byte)
    words = read_data(local_dest)
    dictionary, _ = build_vocab(words, vocab_size, visual_fld)
    index_words = convert_words_to_index(words, dictionary)
    del words           # to save memory
    single_gen = generate_sample(index_words, skip_window)
    
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1])
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(single_gen)
        yield center_batch, target_batch

### Word2Vec

#### Helper functions for word2vec : process_data

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import Counter
import random
import os
import sys
sys.path.append('..')
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf


# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/'
EXPECTED_BYTES = 31344016
DATA_FOLDER = 'data/'
FILE_NAME = 'text8.zip'

def download(file_name, expected_bytes):
    """ Download the dataset text8 if it's not already downloaded """
    file_path = DATA_FOLDER + file_name
    if os.path.exists(file_path):
        print("Dataset ready")
        return file_path
    file_name, _ = urllib.request.urlretrieve(DOWNLOAD_URL + file_name, file_path)
    file_stat = os.stat(file_path)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded the file', file_name)
    else:
        raise Exception('File ' + file_name +
                        ' might be corrupted. You should try downloading it with a browser.')
    return file_path

def read_data(file_path):
    """ Read data into a list of tokens 
    There should be 17,005,207 tokens
    """
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split() 
        # tf.compat.as_str() converts the input into the string
    return words

def build_vocab(words, vocab_size):
    """ Build vocabulary of VOCAB_SIZE most frequent words """
    dictionary = dict()
    count = [('UNK', -1)]
    count.extend(Counter(words).most_common(vocab_size - 1))
    index = 0
    safe_mkdir('processed')
    with open('processed/vocab_1000.tsv', "w") as f:
        for word, _ in count:
            dictionary[word] = index
            if index < 1000:
                f.write(word + "\n")
            index += 1
    index_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, index_dictionary

def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

def generate_sample(index_words, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index, center in enumerate(index_words):
        context = random.randint(1, context_window_size)
        # get a random target before the center word
        for target in index_words[max(0, index - context): index]:
            yield center, target
        # get a random target after the center wrod
        for target in index_words[index + 1: index + context + 1]:
            yield center, target

def get_batch(iterator, batch_size):
    """ Group a numerical stream into batches and yield them as Numpy arrays. """
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1])
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)
        yield center_batch, target_batch

def process_data(vocab_size, batch_size, skip_window):
#     file_path = download(FILE_NAME, EXPECTED_BYTES)
    file_path = DATA_FOLDER + FILE_NAME
    words = read_data(file_path)
    dictionary, _ = build_vocab(words, vocab_size)
    index_words = convert_words_to_index(words, dictionary)
    del words # to save memory
    single_gen = generate_sample(index_words, skip_window)
    return get_batch(single_gen, batch_size)

def get_index_vocab(vocab_size):
    file_path = download(FILE_NAME, EXPECTED_BYTES)
    words = read_data(file_path)
    return build_vocab(words, vocab_size)

In [56]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf


# Model hyperparameters
VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128   #Dimension of the word embedding vector
SKIP_WINDOW = 1    #the context window
NUM_SAMPLED = 64   #the number of negative examples to sample
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 20000
VISUAL_FLD = 'visualization'
SKIP_STEP = 2000

# Paramaters for downloading data
NUM_VISUALIZE = 3000    # number of tokens to visualize

def word2vec(batch_gen):
    """ Build the graph for word2vec model and train it"""
    # step 1 : get the input, output from the dataset
    context_words = tf.placeholder(dtype = tf.int32, shape = [BATCH_SIZE], name = 'context_words')
    target_words = tf.placeholder(dtype = tf.int32, shape = [BATCH_SIZE, 1], name = 'target_words')

    # Step 2 and 3: define the Weights and derive the inference
    # the dimensions of the weight matrix is vocab_size * embed_size
    
    embed_matrix = tf.get_variable(name = 'embed_matrix', shape = [VOCAB_SIZE, EMBED_SIZE], initializer = tf.random_uniform_initializer(minval = -1.0, maxval = 1.0))
    embed = tf.nn.embedding_lookup(embed_matrix, context_words, name = 'embedding')
        
    # Step 4: compute the NCE loss using the sample of the negative labels each time.
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight', shape = [VOCAB_SIZE, EMBED_SIZE], initializer = tf.truncated_normal_initializer(stddev = 1.0 / (EMBED_SIZE ** 0.5)))
        nce_bias = tf.get_variable('nce_bias', initializer = tf.zeros([VOCAB_SIZE]))
        
        # define the loss funcitno to be the NCE loss
        loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weight, biases = nce_bias, labels = target_words, inputs = embed, num_sampled = NUM_SAMPLED, num_classes = VOCAB_SIZE), name = 'loss')
        
    # Step 5: define the optimizer
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
        
    safe_mkdir('checkpoints')
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        total_loss = 0.0 # we use this to calculate the average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('./graphs/no_frills/', sess.graph)
        for index in range(NUM_TRAIN_STEPS):
            centers, targets = next(batch_gen)
            feed_dict = {context_words : centers, target_words : targets}
            _, loss_batch = sess.run([optimizer, loss], feed_dict = feed_dict)
            
            total_loss += loss_batch
            
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                total_loss = 0.0
        writer.close()

def main():
    batch_gen = process_data(VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW)
#     print(next(batch_gen))
    word2vec(batch_gen)
    
if __name__ == '__main__':
    main()

ValueError: ignored

In [36]:
!wget http://mattmahoney.net/dc/text8.zip

--2019-02-15 12:51:03--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   858KB/s    in 36s     

2019-02-15 12:51:39 (849 KB/s) - ‘text8.zip’ saved [31344016/31344016]

